In [4]:
import numpy as np
import cv2
import os
import imageio.v3 as iio
import matplotlib.pyplot as plt
from skimage.feature import hog, local_binary_pattern
from sklearn.metrics.pairwise import cosine_distances

In [5]:
# --- Tiền xử lý ảnh --- 
def preprocess_image(img, img_size=(100, 100)):
    if img.dtype == np.uint8 and img.ndim == 2:
        img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
    if img.ndim == 3:
        img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    if img.ndim != 2:
        raise ValueError("Ảnh không phải 2D sau khi xử lý.")
    img = cv2.equalizeHist(img)
    img = cv2.resize(img, img_size)
    img = img.astype(np.float32) / 255.0
    return img

Assignment 2 (4 điểm)
Đọc dữ liệu ảnh training

In [6]:
# --- Load ảnh train --- 
def load_images_from_folder(folder_path, img_size=(100, 100)):
    print(f"Đang đọc ảnh từ thư mục: {folder_path}")
    images, labels = [], []
    for filename in os.listdir(folder_path):
        if filename.endswith('.gif'):
            img_path = os.path.join(folder_path, filename)
            try:
                img = iio.imread(img_path, index=0)
                img = preprocess_image(img, img_size)
                images.append(img.flatten())
                label = filename.split('.')[0]
                labels.append(label)
                print(f"Đã load ảnh: {filename}")
            except Exception as e:
                print(f"Lỗi khi đọc ảnh {img_path}: {e}")
    return np.array(images), np.array(labels)

In [7]:
# --- Load ảnh test --- 
def load_test_images(folder_path, img_size=(100, 100)):
    print(f"Đang đọc ảnh test từ thư mục: {folder_path}")
    images, labels, filenames = [], [], []
    for filename in os.listdir(folder_path):
        if filename.endswith('.gif'):
            img_path = os.path.join(folder_path, filename)
            try:
                img = iio.imread(img_path, index=0)
                img = preprocess_image(img, img_size)
                images.append(img.flatten())
                label = filename.split('.')[0]
                labels.append(label)
                filenames.append(filename)
                print(f"Đã load ảnh test: {filename}")
            except Exception as e:
                print(f"Lỗi khi đọc ảnh test {img_path}: {e}")
    return np.array(images), np.array(labels), filenames


In [8]:
# --- Trích xuất đặc trưng HOG --- 
def extract_hog(imgs, img_size=(100, 100)):
    print("Đang trích xuất đặc trưng HOG...")
    features = []
    for idx, img in enumerate(imgs):
        img = img.reshape(img_size)
        feature = hog(
            img, orientations=9, pixels_per_cell=(8, 8),
            cells_per_block=(3, 3), block_norm='L2-Hys',
            transform_sqrt=True
        )
        features.append(feature)
    print("Hoàn thành trích xuất HOG.")
    return np.array(features)

In [9]:
# --- Trích xuất đặc trưng LBP --- 
def extract_lbp(imgs, img_size=(100, 100), P=8, R=1):
    print("Đang trích xuất đặc trưng LBP...")
    features = []
    for idx, img in enumerate(imgs):
        img = img.reshape(img_size)
        img_uint8 = (img * 255).clip(0, 255).astype(np.uint8)
        lbp = local_binary_pattern(img_uint8, P, R, method='uniform')
        hist, _ = np.histogram(lbp.ravel(), bins=np.arange(0, P + 3), range=(0, P + 2))
        hist = hist.astype(float)
        hist /= (hist.sum() + 1e-6)
        features.append(hist)
    print("Hoàn thành trích xuất LBP.")
    return np.array(features)

In [10]:
# --- Chuẩn hóa dữ liệu ---
def standardize(X):
    print("Đang chuẩn hóa dữ liệu...")
    return ((X - X.mean(axis=0)) / (X.std(axis=0) + 1e-6)).astype(np.float32)


PCA với 20 chiều

In [11]:
# --- PCA bằng numpy ---
def pca_numpy(X, n_components):
    print(f"Đang thực hiện PCA để giảm xuống {n_components} chiều...")
    mean_vector = X.mean(axis=0)
    X_centered = X - mean_vector
    cov_matrix = np.cov(X_centered, rowvar=False)
    eig_vals, eig_vecs = np.linalg.eigh(cov_matrix)
    sorted_indices = np.argsort(eig_vals)[::-1]
    top_vectors = eig_vecs[:, sorted_indices[:n_components]]
    X_reduced = np.dot(X_centered, top_vectors)
    print("Hoàn thành PCA.")
    return X_reduced, top_vectors, mean_vector


Assignment 3 

In [12]:
# --- Phân loại bằng cosine distance ---
def classify(X_test, X_train, y_train):
    print("Đang phân loại ảnh test...")
    distances = cosine_distances(X_test, X_train)
    preds_idx = np.argmin(distances, axis=1)
    print("Hoàn thành phân loại.")
    return y_train[preds_idx]

 Visualize eigenfaces (cơ sở không gian chính)

In [13]:
# --- Hiển thị eigenfaces ---
def plot_eigenfaces(eigenvectors, n_components=20, img_shape=(100, 100)):
    print(f"Đang vẽ {n_components} eigenfaces...")
    plt.figure(figsize=(15, 6))
    for i in range(n_components):
        plt.subplot(2, 10, i + 1)
        eigenface = eigenvectors[:, i].reshape(img_shape)
        plt.imshow(eigenface, cmap='gray')
        plt.title(f"Eigenface {i + 1}")
        plt.axis('off')
    plt.tight_layout()
    plt.show()
    print("Hoàn thành vẽ eigenfaces.")


Dự đoán và tính độ chính xác

In [ ]:
if __name__ == "__main__":
    base_path = 'C:/Users/ADMIN/Downloads/yalefaces/yalefaces'
    train_path = base_path
    test_path = os.path.join(base_path, 'test')

    X_train_raw, y_train = load_images_from_folder(train_path)
    X_test_raw, y_test, test_filenames = load_test_images(test_path)

    print("Số lượng ảnh train:", X_train_raw.shape[0])
    print("Số lượng ảnh test:", X_test_raw.shape[0])

    X_train_hog = extract_hog(X_train_raw)
    X_train_lbp = extract_lbp(X_train_raw)
    X_train_features = np.hstack((X_train_hog, X_train_lbp))

    X_test_hog = extract_hog(X_test_raw)
    X_test_lbp = extract_lbp(X_test_raw)
    X_test_features = np.hstack((X_test_hog, X_test_lbp))

    X_train_std = standardize(X_train_features)
    X_test_std = standardize(X_test_features)

    # --- Chỉnh số lượng PCA phù hợp ---
    n_components = min(20, X_train_std.shape[0]-1)
    X_train_pca, eigenvectors, mean_vector = pca_numpy(X_train_std, n_components=n_components)
    X_test_centered = X_test_std - mean_vector
    X_test_pca = np.dot(X_test_centered, eigenvectors)

 


    # --- PCA trên ảnh gốc để hiển thị eigenfaces ---
    X_train_img_std = standardize(X_train_raw)
    _, eigenvectors_img, _ = pca_numpy(X_train_img_std, n_components=20)
    plot_eigenfaces(eigenvectors_img, n_components=20, img_shape=(100, 100))

    # --- Phân loại ---
    predictions = classify(X_test_pca, X_train_pca, y_train)


Đang đọc ảnh từ thư mục: C:/Users/ADMIN/Downloads/yalefaces/yalefaces
Đang đọc ảnh test từ thư mục: C:/Users/ADMIN/Downloads/yalefaces/yalefaces\test
Đang trích xuất đặc trưng HOG...
Hoàn thành trích xuất HOG.
Đang trích xuất đặc trưng LBP...
Hoàn thành trích xuất LBP.
Đang trích xuất đặc trưng HOG...
Hoàn thành trích xuất HOG.
Đang trích xuất đặc trưng LBP...
Hoàn thành trích xuất LBP.
Đang chuẩn hóa dữ liệu...
Đang chuẩn hóa dữ liệu...
Đang thực hiện PCA để giảm xuống 20 chiều...


C:\Users\ADMIN\AppData\Local\Temp\ipykernel_19900\4273975951.py:4: RuntimeWarning: Mean of empty slice.
  return ((X - X.mean(axis=0)) / (X.std(axis=0) + 1e-6)).astype(np.float32)
C:\Users\ADMIN\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
C:\Users\ADMIN\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\numpy\core\_methods.py:206: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
C:\Users\ADMIN\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\numpy\core\_methods.py:163: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean,
C:\User

LinAlgError: 0-dimensional array given. Array must be at least two-dimensional

In [ ]:
  # --- Accuracy ---
accuracy = np.mean(predictions == y_test)
print(f"Độ chính xác nhận diện khuôn mặt: {accuracy * 100:.2f}%")